<img src='../img/logo.png' alt='DS Market logo' height='150px'>

# Clustering

## Table of Contents

* [A. Introduction](#introduction)
* [B. Importing Libraries](#libraries)
* [C. Importing data](#data)
* [D. Products clustering](#products)
* [D. Products clustering](#products)

## A. Introduction <a class="anchor" id="introduction"></a>

The goal of this notebook is to identify groups of products that behave in a similar way, so DS Market can evaluate the performance of their different campaigns. We will provide those groups and the ideal number of these.

Additionally, we will try to identify how similar are stores from one another in case it makes any sense to do it.

## B. Importing Libraries <a class="anchor" id="libraries"></a>

In [2]:
# system and path management
import sys
sys.path.append('../scripts') # including helper functions inside the scripts folder

# removing system warnings
import warnings
warnings.filterwarnings('ignore')

# data manipulation
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt

# plotting options
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (10, 7)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.2f}'.format

# helper functions
import outlier_management as outliers
import file_management

## C. Importing Data <a class="anchor" id="data"></a>

In [3]:
# downloading the processed data files from gdrive
directory = '../data/processed/'
urls = [
    {'filename': 'sales_processed.csv', 'url': 'https://drive.google.com/file/d/1JdeAgraKcaFQJrjG2HPVb5D0VD0iTlNB/view?usp=sharing'},
    {'filename': 'prices_processed.csv', 'url': 'https://drive.google.com/file/d/1pSEJAQfAU-owDjKmxcPrxf3CpGFivwa6/view?usp=sharing'},
    {'filename': 'calendar_processed.csv', 'url': 'https://drive.google.com/file/d/1Lnji96iBkTpFiWo-QXeW3TvESiNYWCML/view?usp=sharing'}
]
        
file_management.download_files_from_url(urls, directory)

sales = pd.read_csv(directory + 'sales_processed.csv')
prices = pd.read_csv(directory + 'prices_processed.csv')
calendar = pd.read_csv(directory + 'calendar_processed.csv')

sales_processed.csv file already exists in ../data/processed/
prices_processed.csv file already exists in ../data/processed/
calendar_processed.csv file already exists in ../data/processed/


In [4]:
# downloading the feature file from gdrive
directory = '../data/features/'
urls = [
    {'filename': 'master.csv', 'url': 'https://drive.google.com/file/d/1_OCpE6AZK3ju5RJVTJZm8ox5Cn8G3_Ag/view?usp=sharing'},
]
        
file_management.download_files_from_url(urls, directory)

master = pd.read_csv(directory + 'master.csv')

master.csv file already exists in ../data/features/


____

Popularity of products

- plot by number of sales by month /week distribution(avg sales in last month...)
- filter by city
- filter by store
- monthly increment of sales (?)

clustering:

- scatter plot... 1v1 (with num variables)
- what caracterises a store? amount of sales they have? Num of products they have...
